### 1. import package

In [ ]:
import requests
import pandas as pd
import re
import time

from selenium import webdriver
from bs4 import BeautifulSoup

### 2. set wait time

In [ ]:
wait_time = 1

### 3. functions

In [ ]:
def make_url(category, date, page):
    url = "http://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1=" + str(category) + "#&date=" + date + " 00:00:00&page=" + str(page)
    return url

In [ ]:
def last_page(url):
    
    driver =  webdriver.PhantomJS()
    driver.get(url)

    time.sleep(wait_time)
    pages = driver.find_elements_by_css_selector("#paging ._paging")
    total_page = len(pages)
    
    if len(pages) > 9:
        
        pages[9].click()        
        time.sleep(wait_time)
        pages = driver.find_elements_by_css_selector("#paging ._paging")
        total_page += len(pages) - 1
        
        while len(pages) > 10:
        
            pages[10].click()        
            time.sleep(wait_time)
            pages = driver.find_elements_by_css_selector("#paging ._paging")
            
            if len(pages) > 10:
                total_page += len(pages) - 1 
            else:
                total_page += len(pages)
            
    driver.close()   
    
    return total_page

In [ ]:
def last_page(url):
    
    driver =  webdriver.PhantomJS()
    driver.get(url)

    time.sleep(wait_time)
    pages = driver.find_elements_by_css_selector("#paging ._paging")
    total_page = len(pages)
    
    if len(pages) > 9:
        
        pages[9].click()        
        time.sleep(wait_time)
        pages = driver.find_elements_by_css_selector("#paging ._paging")
        total_page += len(pages) - 1
        
        while len(pages) > 10:
        
            pages[10].click()        
            time.sleep(wait_time)
            pages = driver.find_elements_by_css_selector("#paging ._paging")
            
            if len(pages) > 10:
                total_page += len(pages) - 1 
            else:
                total_page += len(pages)
            
    driver.close()   
    
    return total_page

In [ ]:
# using json
def get_likeit(aid, oid):    
    url = "http://news.like.naver.com/likeIt/likeItContent.jsonp?_callback=window.__jindo2_callback._7105&serviceId=NEWS&displayId=NEWS&contentsId=ne_" + str(oid) + "_" + str(aid) + "&lang=ko&viewType=recommend"
    response = requests.get(url)
    return response.text.split('likeItCount":')[1].split(",")[0]
    
# using bs4
def get_content(path):
    
    response = requests.get(path)
    dom = BeautifulSoup(response.content, "html.parser")

    if len(dom.select("#articleTitleCommentCount .lo_txt")) == 0:
        return 0, 0, "-"
    
    comment = dom.select_one("#articleTitleCommentCount .lo_txt").text
    content = dom.select_one("#articleBodyContents").text.replace("\n","").replace("\r","").replace("\t","")
    aid = path.split("aid=")[1]
    oid = path.split("oid=")[1].split("&")[0]
    likeit = get_likeit(aid, oid)
    
    return comment, likeit, content

url = "http://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=105&oid=023&aid=0003117328"
get_content(url)

In [ ]:
def article_url_list(url):
    
    driver =  webdriver.PhantomJS()
    driver.get(url)
    
    time.sleep(wait_time)
    articles = driver.find_elements_by_css_selector("#section_body.section_body ul li")
    
    article_df = pd.DataFrame(columns=["newsid", "newspaper", "title", "link", "comment", "likeit", "content"])
    
    for article in articles:
        link = article.find_element_by_css_selector("a").get_attribute("href")
        title = article.find_element_by_css_selector("a").get_attribute("title")
        newspaper = article.find_element_by_css_selector("span").text
        newsid = link.split("aid=")[1]
        comment, likeit, content = 0,0,"-"
        
        tmp_dict = {
            "newsid": newsid, 
            "newspaper": newspaper, 
            "title": title, 
            "link": link, 
            "comment": comment, 
            "likeit": likeit, 
            "content": content,
        }
        
        article_df.loc[len(article_df)] = tmp_dict
        
    article_df["date"] = url.split("date=")[1].split(" ")[0]
    article_df["category"] = url.split("sid1=")[1].split("#")[0]
    
    driver.close()
    
    return article_df

In [ ]:
def date_category_df(category, date):
    
    url = make_url(category, date, 1)
    last_page_number = last_page(url)
    
    result_df = pd.DataFrame(columns=["newsid", "newspaper", "title", "link", "comment", "likeit", "content", "date", "category"])

    for page in range(1, last_page_number + 1):
        link = make_url(category, date, page)
        tmp_df = article_url_list(link)        
        result_df = pd.concat([result_df, tmp_df])
        print(last_page_number, page)
        
    return result_df

### 4. crawling news list

In [ ]:
def crawling_news_list():
    
    for category in range(100,106):
        result = main(category,"2016-01", 31)
        result_df = result.reset_index(drop=True)
        result_df.to_csv("./datas/" + str(category) + "_2016-01.csv", index=False, encoding="utf-8")

    for category in range(100,106):
        result = main(category,"2016-02", 28)
        result_df = result.reset_index(drop=True)
        result_df.to_csv("./datas/" + str(category) + "_2016-02.csv", index=False, encoding="utf-8")

    for category in range(100,106):
        result = main(category,"2016-03", 31)
        result_df = result.reset_index(drop=True)
        result_df.to_csv("./datas/" + str(category) + "_2016-03.csv", index=False, encoding="utf-8")

    for category in range(100,106):
        result = main(category,"2016-04", 30)
        result_df = result.reset_index(drop=True)
        result_df.to_csv("./datas/" + str(category) + "_2016-04.csv", index=False, encoding="utf-8")

    for category in range(100,106):
        result = main(category,"2016-05", 31)
        result_df = result.reset_index(drop=True)
        result_df.to_csv("./datas/" + str(category) + "_2016-05.csv", index=False, encoding="utf-8")

    for category in range(100,106):
        result = main(category,"2016-06", 30)
        result_df = result.reset_index(drop=True)
        result_df.to_csv("./" + str(category) + "_2016-06.csv", index=False, encoding="utf-8")

### 5. crawling contents in news list

In [ ]:
def set_content(path):
    df = pd.read_csv(path)
    df_size = len(df)
    
    for idx, row in df.iterrows():
        
        if df.loc[idx, "content"] == "-":
            comment, likeit, content = get_content(row.link)
            df.loc[idx, "comment"] = comment
            df.loc[idx, "likeit"] = likeit
            df.loc[idx, "content"] = content

            df.to_csv(path, index=False, encoding="utf-8")

            print(df_size, idx, likeit, comment, len(content), row.link)
        

path = "./datas/105_2016-01.csv"
set_content(path)    

In [ ]:
path = "./datas/105_2016-01.csv"
df = pd.read_csv(path)
df[160:180]

In [ ]:
def merge(category):
    
    df_list = []
    
    for month in range(1,7):
        link_str = "./news_data/" + str(category) + "_2016-0" + str(month) + ".csv"

        df = pd.read_csv(link_str)
        df_list.append(df)
        
        print(link_str)

    df = pd.concat(df_list).reset_index(drop=True)
    
    df.to_csv("./news_data/" + str(category) + ".csv")
    
# for category in range(100, 106):
#     merge(category)
    